## Exercise 8.3

If you did the exercises in Chapter 3, you saw the effect of the Hamming window, and some of the other windows provided by NumPy, on spectral leakage.  We can get some insight into the effect of these windows by looking at their DFTs.

In addition to the Gaussian window we used in this window, create a Hamming window with the same size.  Zero pad the windows and plot their DFTs.  Which window acts as a better low-pass filter?  You might find it useful to plot the DFTs on a log-$y$ scale.

Experiment with a few different windows and a few different sizes.

In [1]:
import os, numpy as np, matplotlib.pyplot as plt, scipy.signal
from thinkdsp import decorate, TriangleSignal
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
if not os.path.exists('thinkdsp.py'):
    !wget https://github.com/cpulover/signal-processing/blob/master/thinkdsp.py

A sample wave:

In [2]:
signal = TriangleSignal(freq=500)
wave = signal.make_wave(duration=1.0, framerate=15943)

Create different windows:

In [3]:
M = 20
std = 3

gaussian = scipy.signal.gaussian(M=M, std=std)   
bartlett = np.bartlett(M)
blackman = np.blackman(M)
hamming = np.hamming(M)
hanning = np.hanning(M)
#beta = 5
#kaiser = np.kaiser(M,beta)

windows = [blackman, gaussian, hanning, hamming]
names = ['blackman', 'gaussian', 'hanning', 'hamming']

for window in windows:
    window /= sum(window) #normalize

Alternatively, we can generalize different windows by using Kaiser window with different shape parameter (beta):

In [4]:
def plot_windows(beta):
    kaiser = np.kaiser(M,beta)
    kaiser /= sum(kaiser)
    plt.plot(kaiser, label='kaiser')
    for window, name in zip(windows, names):
        plt.plot(window, label=name)
    decorate(xlabel='Time')

In [5]:
slider = widgets.FloatSlider(min=0.1, max=10, value=4)
interact(plot_windows, beta=slider);

interactive(children=(FloatSlider(value=4.0, description='beta', max=10.0, min=0.1), Output()), _dom_classes=(…

Notice that higher the beta is, narrower the curve is. The gaussion, blackman, hanning, hamming curve is similar to the kaiser curve with beta of 9.8, 8.6, 6, 5, respectively.

 Define function to zero pad the windows and plot their DFTs:

In [6]:
def zero_pad(array, n):
    """Extends an array with zeros.
    array: NumPy array
    n: length of result
    returns: new NumPy array
    """
    res = np.zeros(n)
    res[:len(array)] = array
    return res

In [20]:
def plot_window_dfts(beta):
    kaiser = np.kaiser(M,beta)
    kaiser /= sum(kaiser)
    padded_kaiser = zero_pad(kaiser, len(wave))
    plt.plot(abs(np.fft.rfft(padded_kaiser)), label='kaiser')
    for window, name in zip(windows, names):
        padded =  zero_pad(window, len(wave))
        dft_window = np.fft.rfft(padded)
        plt.plot(abs(dft_window), label=name)
        decorate(xlabel='Frequency (Hz)',yscale='log')

In [21]:
slider = widgets.FloatSlider(min=0.1, max=10, value=4)
interact(plot_window_dfts, windows=windows,names=names,beta=slider);

interactive(children=(FloatSlider(value=4.0, description='beta', max=10.0, min=0.1), Output()), _dom_classes=(…

Notice that higher the beta is, faster the DFT drops off and less persistent sidelobes it has. Therefore, the Hanning window might have the best combination of fast drop off and minimal sidelobes.